In [1]:
!pip install opendatasets


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
dataset = 'https://www.kaggle.com/datasets/stackoverflow/pythonquestions'
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  hyeongchanim


Your Kaggle Key:

  ········


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


100% 558M/558M [00:40<00:00, 14.5MB/s] 

In [2]:
!pip install transformers

  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Using cached regex-2023.3.23-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (771 kB)
  Using cached filelock-3.10.7-py3-none-any.whl (10 kB)
  Using cached tokenizers-0.13.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)
  Using cached huggingface_hub-0.13.3-py3-none-any.whl (199 kB)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


## Data Preprocessing
* CSV 파일을 Pandas DataFrame으로 로드합니다.
* 질문과 답변을 합쳐서 하나의 문서로 만듭니다.
* 문서를 전처리합니다. 예를 들어, HTML 태그나 코드 블록을 제거하거나 소문자로 변환합니다.
* 문서를 토큰화합니다. 예를 들어, 문장부호나 공백을 기준으로 문장을 나누고 단어를 추출합니다.
* 불용어를 제거합니다. 즉, 자주 등장하지만 의미를 가지지 않는 단어를 제거합니다.

In [1]:
import pandas as pd

# CSV 파일을 Pandas DataFrame으로 load
questions_df = pd.read_csv("./pythonquestions/Questions.csv", encoding='latin1')
answers_df = pd.read_csv("./pythonquestions/Answers.csv", encoding='latin1')
tags_df = pd.read_csv("./pythonquestions/Tags.csv", encoding='latin1')

In [2]:
# 질문과 답변을 합쳐서 하나의 문서로 만듭니다.
docs = []
for index, row in questions_df.iterrows():
    question = row["Title"] + " " + row["Body"]
    answer = answers_df.loc[answers_df["ParentId"] == row["Id"], "Body"].values
    if len(answer) > 0:
        answer = answer[0]
        doc = question + " " + answer
        docs.append(doc)
        
        
"""
questions_df 데이터프레임의 Title과 Body 컬럼을 합쳐서 질문을 만들고, 
해당 질문에 대한 답변을 answers_df 데이터프레임에서 찾아서 합친다.
이렇게 만들어진 문서는 docs 리스트에 추가한다.
"""

'\nquestions_df 데이터프레임의 Title과 Body 컬럼을 합쳐서 질문을 만들고, \n해당 질문에 대한 답변을 answers_df 데이터프레임에서 찾아서 합친다.\n이렇게 만들어진 문서는 docs 리스트에 추가한다.\n'

In [8]:
import string
import nltk

nltk.download('punkt')  # punkt 모듈 다운
nltk.download('wordnet') # wordnet 모듈 다운

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
"""
문서 전처리
"""

import re
import string

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


# 문서를 전처리합니다.
def preprocess_doc(doc):
    # HTML 태그를 제거합니다.
    doc = re.sub('<[^>]*>', '', doc)

    # 코드 블록을 제거합니다.
    doc = re.sub('```\w*\n([\s\S]*?)```', '', doc)

    # 소문자로 변환합니다.
    doc = doc.lower()

    # 문장부호를 제거합니다.
    doc = doc.translate(str.maketrans('', '', string.punctuation))

    # 단어를 토큰화합니다.
    words = word_tokenize(doc)

    # 불용어를 제거합니다.
    # 불용어란 문장에서 자주사용되나 의미분석에 영향X
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]

    # 단어를 원형으로 변환합니다.
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]

    # 단어를 공백으로 구분하여 문서로 만듭니다.
    doc = ' '.join(words)

    return doc


# 모든 문서를 전처리합니다.
preprocessed_docs = []
for doc in docs:
    preprocessed_doc = preprocess_doc(doc)
    preprocessed_docs.append(preprocessed_doc)


## Model Architecture Compose
* 모델 아키텍처는 Transformer 기반의 언어 모델을 사용
* Huggingface의 GPT-2 사용

Token indices sequence length is longer than the specified maximum sequence length for this model (1138 > 1024). Running this sequence through the model will result in indexing errors

In [37]:
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

# GPU가 사용 가능한지 확인합니다.
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3090


In [42]:
# train set으로 사용할 텍스트 파일을 생성합니다.
with open('trainset.txt', 'w', encoding='utf-8') as f:
    for doc in doc:
        f.write(doc + '\n')

# trainset 파일을 불러와서 TextDataset과 DataCollatorForLanguageModeling을 사용하여 train set을 처리합니다.
from transformers import TextDataset, DataCollatorForLanguageModeling

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='./trainset.txt',
    block_size=block_size,
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [44]:
# GPT2 tokenizer를 load합니다.
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# max_length, block_size(GPU 메모리에 맞춰서)를 설정합니다.
max_length = 128
block_size = 128

# 전처리된 문서를 tokenizing합니다.
tokenized_docs = [tokenizer(doc, max_length=max_length, truncation=True) for doc in preprocessed_docs]

In [45]:
# TextDataset과 DataCollatorForLanguageModeling을 사용하여 train set을 처리합니다.
from transformers import TextDataset, DataCollatorForLanguageModeling

file_path = "./trainset.txt"  # train set의 파일 경로를 지정해 주세요
dataset = TextDataset(file_path=file_path, tokenizer=tokenizer, block_size=block_size)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)



# 모델을 load합니다.
model = GPT2LMHeadModel.from_pretrained('gpt2')

# GPU로 모델을 옮겨줍니다.
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [63]:
# TrainingArguments를 설정합니다.
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=500,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    save_steps=10_000,              # number of updates steps between two checkpoints
    save_total_limit=2,             # limit the total amount of checkpoints, delete the oldest checkpoints
    prediction_loss_only=True,      # only compute loss for masked tokens
    learning_rate=2e-5,             # learning rate for optimizer
    warmup_steps=500,               # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # number of updates steps to accumulate before performing a backward/update pass
    weight_decay=0.01,              # strength of weight decay
    logging_dir='./logs',           # directory for storing logs
    logging_steps=10_000,           # log & save weights every logging_steps
)

In [64]:
# Trainer 객체를 생성하고 학습을 시작합니다.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)
trainer.train()


Step,Training Loss


TrainOutput(global_step=500, training_loss=0.004153316497802735, metrics={'train_runtime': 62.6332, 'train_samples_per_second': 79.83, 'train_steps_per_second': 7.983, 'total_flos': 326615040000000.0, 'train_loss': 0.004153316497802735, 'epoch': 500.0})

## 모델 평가 및 예측수행
* 모델 저장 후 모델에 대한 예측수행

In [65]:
# 모델 저장
model.save_pretrained('./saved_model')

# Model 작동시켜보기

1. 모델 불러오기

In [66]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('./saved_model')

2. 모델을 사용하여 원하는 문장에 대한 답변을 생성하는 코드를 작성합니다. 
* 이때 generate 메소드를 사용합니다. generate 메소드는 생성할 문장의 길이와 다양한 설정 등을 인자로 받습니다. 
* 예를 들어, 아래 코드에서는 generate 메소드의 인자로 prompt와 max_length를 전달합니다.

In [74]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

while True:
    prompt = input("\n Enter a question (or 'Quit' to exit): ")
    if prompt == 'Quit':
        break

    encoded_prompt = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
    output_sequences = model.generate(
        input_ids=encoded_prompt,
        max_length=512,
        temperature=0.7,
        top_k=0,
        top_p=0.9,
        repetition_penalty=1.0,
        do_sample=True,
        num_return_sequences=1,
    )

    generated_sequence = output_sequences[0].tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
    print(text)


 Enter a question (or 'Quit' to exit):  What is the python?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is the python?

The python is a project of Python programming language (PL) that was developed by the team of Rob Cordain, who has been working on Python since 2003. The project has since grown to be a top-notch software development platform for Python developers and is used by more than 100,000 users worldwide.

The python is a python that is based on the Python programming language. It is based on the framework of the Jupyter notebook, which is a Python-based data type system. It is based on the python-2.5 module, which was released in the latest version of Python 3.6. The python is an open-source project. The project uses a mix of Python libraries to implement various features such as data structures, user interface, and more.

What is the python interpreter?

The python interpreter is a Python programming language. The python interpreter is a Python object-oriented programming language (ODL). The python interpreter is based on the Python programming language and is used by ove


 Enter a question (or 'Quit' to exit):  How can I install python3 in linux environment?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

What is the difference between list and tuple?

List and tuple are two different concepts that are completely different concepts. List and tuple are a more complex way of thinking about lists and tuples. For example, consider the following example:

class Foo { public: Foo(int i); };

The above example defines a Foo class that has a Foo. In the example below, the Foo object is a member of Foo. The definition of Foo.Literal does not include the fact that the Foo is a literal. The definition of Foo.Literal is used to represent the type of the object. In this case, the definition of Foo.Literal is:

// Foo.Literal is a literal public: Foo(int i); // Foo.Literal is a literal public: Foo(int i);

The definition of Foo.Literal is:

// Foo.Literal is a literal

What are the differences between tuple and list?

List and tuple are a more complex way of thinking about lists and tuples. For example, consider the following example:

class Foo { public: Foo(int i); };

The above example defines a F


 Enter a question (or 'Quit' to exit):  How to use dictionary and set in python3?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How to use dictionary and set in python3?

First we need to define dictionary.py for our dictionary. Then we need to create a dictionary.py file asthma.txt and save it to /etc/python3/dict.py.

import dict import time def __init__ ( self ): # create our dictionary self. dictionary. set_dict ( self. dict ) def __init__ ( self, age ): # set the age of the dictionary self. age = age self. set_dict ( dict ) def get_dict ( self ): # get the dictionary from dictionary self. __dict__ = dict def get_dict ( self ): # get the dictionary from the specified age self. age = age self. set_dict ( dict ) def get_dict ( self ): # get the dictionary from the specified age self. age = age

Once the dictionary is created, we can start writing the dictionary.py file.

def _dict_file ( self ): """ Get dictionary from the specified age """ def __init__ ( self ): """ Create the dictionary """ def get_dict ( self ): """ Get the dictionary from the specified age """ self. age = age self. set_dict ( dict ) def g


 Enter a question (or 'Quit' to exit):  Quit
